# Setting Up Work Environment

In [2]:
# Install required libraries
!pip install --upgrade google-generativeai                 # Install Google Generative AI (Gemini) SDK
!pip install -q -U google-genai                            # Another variant of Google Generative AI SDK
!pip install langchain-community                           # Community-supported LangChain tools
!pip install docarray                                      # Used for storing and searching documents in memory
!pip install -U langchain-google-genai                     # LangChain integration for Google Generative AI (Gemini)
!pip install wikipedia
!pip install langchain_experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.4 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.18
    Uninstalling google-ai-generativelanguage-0.6.18:
      Successfully uninstalled google-ai-generativelanguage-0.6.18
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-google-genai 2.1.6 requires google-ai-generativelanguage<0.7.0,>=0.6.18, but you have google-ai-generativelanguage 0.6.15 which is incompatible.


  Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl.metadata (9.8 kB)
Using cached google_ai_generativelanguage-0.6.18-py3-none-any.whl (1.4 MB)
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 4.9 MB/s eta 0:00:00


In [24]:
import os
from google import genai                              # Google GenAI main module
import google.generativeai as ggenai                  # Another way to access the GenAI functionality
from google.colab import userdata                     # For securely retrieving secrets in Colab

# LangChain integrations with Gemini
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings

import os
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool, PythonREPL
from langchain.agents import tool
from datetime import date

In [2]:
# Get the Gemini API key stored securely in Colab
key = userdata.get('genai_api')

# Instantiate a GenAI client with the key
client = genai.Client(api_key=key)

# Configure the generative AI with the same key for further calls
ggenai.configure(api_key=key)

List the set of available models

In [3]:
models = ggenai.list_models()
# You can uncomment below to print model names
# for model in models:
#     print(model.name)

In [13]:
#  Initialize the Gemini LLM (Chat Model)
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",     # Choose your preferred Gemini model (e.g., flash, pro, etc.)
    temperature=0.0,              # Set to 0 for deterministic output
    google_api_key=key
)

We will begin by loading two tools: the LLM Math Tool and the Wikipedia Tool.

The LLM Math Tool is a chain that integrates a language model with a calculator component, enabling it to solve mathematical problems by interpreting queries and performing calculations as needed.

The Wikipedia Tool functions as an API wrapper that connects to the Wikipedia platform. It allows users to issue search queries and retrieve relevant content directly from Wikipedia’s knowledge base. These tools will enable the language model to perform both computational reasoning and information retrieval tasks effectively.

In [14]:
tools = load_tools(["llm-math","wikipedia"], llm=llm)

Next, we will initialize the agent by specifying the necessary components: the set of tools, the language model, and the agent type. In this case, we will use the AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, which is specifically optimized for use with chat-based language models.

We will also enable handle_parsing_errors=True. This setting is particularly useful when the language model produces output that cannot be parsed into a valid action and action_input—the expected response format. When such parsing issues occur, the misformatted output is returned to the language model, prompting it to revise its response.

Finally, we will set verbose=True to enable detailed logging, which provides visibility into each step of the agent's decision-making and execution process.

In [15]:
agent= initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

# Constructing a Math Tutor Agent

We will begin by presenting the agent with a basic math question: “What is 25% of 300?” While this is a relatively straightforward calculation, it serves as a useful example for understanding the internal reasoning process and tool usage of the agent. Observing how the agent interprets and responds to this question will provide insight into its underlying mechanisms.

In [16]:
agent("What is the 25% of 300?")



> Entering new AgentExecutor chain...
Thought: To find 25% of 300, I need to multiply 300 by 0.25. I can use the calculator to do this.
Action:
```json
{
  "action": "Calculator",
  "action_input": "300 * 0.25"
}
```
Observation: Answer: 75.0
Thought:I now know the final answer
Final Answer: 75.0

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75.0'}

In [17]:
agent("If you have a rectangular garden with dimensions 12 meters by 8 meters, what is the total area of the garden in square meters?")



> Entering new AgentExecutor chain...
Thought: To find the area of a rectangle, I need to multiply the length by the width. In this case, the length is 12 meters and the width is 8 meters.
Action:
```json
{
  "action": "Calculator",
  "action_input": "12 * 8"
}
```
Observation: Answer: 96
Thought:I have calculated the area by multiplying the length and width.
Final Answer: 96

> Finished chain.


{'input': 'If you have a rectangular garden with dimensions 12 meters by 8 meters, what is the total area of the garden in square meters?',
 'output': '96'}

Upon entering the Agent Executor Chain, the agent begins by reasoning through the task. It first generates a thought, followed by an action, which is represented as a JSON object. This object consists of two key components:

- `action`: Specifies the tool to be used (in this case, `the calculator`).

- `action_input`: Represents the input passed to the selected tool (e.g., `"12*8"`).

Subsequently, an observation is produced—this is the output returned by the tool. In this example, the calculator tool evaluates the expression and returns the result `96`. If visualized in a live code environment, this observation may appear in a different color to distinguish it from the agent’s thought and action steps. This sequence illustrates how the agent decomposes the task, selects an appropriate tool, executes the action, and integrates the resulting output into its response.

# Constructing a Wikipedia Search Agent

The next agent we will examine utilizes the Wikipedia API for information retrieval. In this example, we will pose a question regarding Andrew Ng. By enabling intermediate steps, we can observe the agent’s internal reasoning and execution process—specifically how it formulates the query, selects the appropriate tool, interacts with the Wikipedia API, and processes the response to generate the final answer. This provides insight into the agent’s step-by-step decision-making when handling knowledge-based queries.

In [18]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question)



> Entering new AgentExecutor chain...
Thought: I need to find out what book Tom M. Mitchell wrote. I can use Wikipedia to search for information about him.
Action:
```json
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```

/usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.11/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

# Constructing a Python Programming Assistant Agent

In the next example, we will build a Python Programming Assistant Agent. This type of agent is conceptually similar to tools like GitHub Copilot or ChatGPT with the Code Interpreter plugin, where a language model is used to generate code, execute it, and interpret the results.

We will begin by creating a Python agent using the same language model previously configured. This agent will be equipped with the Python REPL Tool, which provides a runtime environment for executing Python code interactively.

Once the agent generates a code snippet, it will execute that code using the REPL. The resulting output will then be returned to the agent, enabling it to decide on the next course of action based on the computed result. This feedback loop allows the agent to iteratively reason and solve programming tasks.

In [20]:
agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [21]:
customer_list = [["Harrison", "Chase"],
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"],
                 ["Geoff","Fusion"],
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [22]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")

/tmp/ipython-input-22-2161092319.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run(f"""Sort these customers by \




> Entering new AgentExecutor chain...
I need to sort the list of customers first by last name and then by first name. I can use the `sorted` function with a custom key to achieve this. The key should be a tuple containing the last name and first name.
Action: [Python_REPL]
Action Input: ```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]

sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))

print(sorted_customers)
```
Observation: [Python_REPL] is not a valid tool, try one of [Python_REPL].
Thought:I apologize for the error. I meant to use the Python REPL to execute the code.
Action: [Python_REPL]
Action Input: ```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]

sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))

print(sorted_customers)
```
O

"[['Jen', 'Ayai'], ['Lang', 'Chain'], ['Harrison', 'Chase'], ['Elle', 'Elem'], ['Trance', 'Former'], ['Geoff', 'Fusion'], ['Dolly', 'Too']]"

In [23]:
import langchain
langchain.debug=True
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""")
langchain.debug=False

[chain/start] [chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [chain:AgentExecutor > chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [chain:AgentExecutor > chain:LLMChain > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou have access to a python REPL, which you can use to execute python code.\nI

# Constructing Your Customized & Own Agent

In the previous examples, we utilized tools and agents that are already predefined within the LangChain framework. However, one of the most powerful features of LangChain agents is their ability to integrate with custom sources of information, including proprietary APIs, internal databases, and other data services.

In this section, we will demonstrate how to create a custom tool, allowing you to connect an agent to virtually any resource of your choice.

As a simple example, we will build a tool that returns the current date. To do this, we begin by importing the `@tool` decorator. This decorator can be applied to any Python function, transforming it into a LangChain-compatible tool that agents can invoke during execution.

We will define a function named `time`, which accepts a single input—a text string. Although this parameter is required for the function signature, it will not be used in this specific implementation. The function will return the current date by calling the `datetime` module.

In addition to defining the function, we will provide a detailed docstring. This documentation is essential, as it guides the agent in understanding when and how to invoke the tool. For instance, in this case, we explicitly state that the input should always be an empty string, since the function does not utilize the input parameter.

If the tool were to require more specific inputs—such as a search query or an **SQL statement** those requirements should be clearly stated in the docstring to ensure the agent can interact with the tool correctly and reliably.

In [25]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [26]:
agent = initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

**`Note:`**

The agent will sometimes come to the wrong conclusion (agents are a work in progress!).

If it does, please try running it again.

In [27]:
try:
    result = agent("whats the date today?")
except:
    print("exception on external access")



> Entering new AgentExecutor chain...
Thought: I need to use the time tool to get today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```
Observation: 2025-07-06
Thought:I have the date.
Final Answer: 2025-07-06

> Finished chain.
